In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

In [6]:
df.head(50)

,BTC_open,BTC_close,BTC_volume,DOT_open,DOT_close,DOT_volume,perc_BTC_change_1hr,perc_BTC_change_2hr,perc_BTC_change_3hr,perc_BTC_change_4hr,perc_BTC_change_5hr,perc_BTC_change_6hr,perc_DOT_change_1hr,perc_DOT_change_2hr,perc_DOT_change_3hr,perc_DOT_change_4hr,perc_DOT_change_5hr,perc_DOT_change_6hr,lag_time,met_threshold
time,,,,,,,,,,,,,,,,,,,,
2021-01-29 08:00:00,32323.72,36388.23,4.579055e+08,16.290,16.750,2493213.70,12.574388,11.828902,10.118670,9.862769,7.601011,6.820023,2.823818,2.760736,3.140394,2.258852,0.600601,-1.296405,0 days 04:00:00,1.0
2021-02-08 12:00:00,39459.41,43233.85,7.173601e+08,22.290,22.010,5207569.47,9.565374,9.559654,10.871974,10.242315,9.982707,10.710966,-1.256169,-0.136116,6.431335,9.502488,9.502488,9.339295,0 days 00:00:00,1.0
2021-02-23 20:00:00,45507.19,47810.01,2.587453e+08,31.000,32.680,4951864.52,5.060343,2.056434,1.422008,1.080340,-1.632396,1.803114,5.419355,2.799623,1.333333,-1.625527,-3.227717,1.616915,0 days 04:00:00,1.0
2021-05-19 16:00:00,37451.48,39735.17,8.613382e+08,27.120,28.990,18342311.92,6.097730,5.710329,10.336180,14.969926,3.079451,0.957870,6.895280,4.393230,4.884226,16.099319,-19.315335,-27.160804,0 days 01:00:00,1.0
2022-05-11 13:00:00,29453.96,31142.09,3.548136e+08,8.458,10.230,16375906.85,5.731419,-1.193060,-2.241606,-1.913009,-0.829802,1.858948,20.950579,-1.917546,-6.232814,-7.754734,-7.336957,-2.385496,0 days 04:00:00,1.0
2022-05-12 07:00:00,26433.10,27883.64,5.873600e+08,7.518,7.939,8113107.91,5.487589,3.644864,0.712298,-1.543809,-2.878841,-4.249129,5.599894,4.077084,-0.613420,-3.687978,-7.878858,-12.276243,0 days 03:00:00,1.0
2022-06-30 23:00:00,18873.41,19908.49,1.774439e+08,6.782,7.060,1111699.24,5.484330,5.908561,6.270942,5.429809,5.037819,4.321578,4.099086,4.903418,5.325973,3.899926,4.191263,3.171124,0 days 02:00:00,1.0
2022-11-10 13:00:00,16485.87,17546.72,2.128704e+08,5.669,6.040,2362482.65,6.434905,7.979417,7.680034,4.905221,3.546129,4.745969,6.544364,8.030764,8.399139,6.002106,5.483758,5.299861,0 days 03:00:00,1.0


In [4]:
df = pd.read_csv('../data/ready/DOT.csv',index_col='time').drop('Unnamed: 0', axis = 1).dropna()
df.shape

(8, 20)

In [5]:
df['met_threshold'].value_counts(normalize=True)

1.0    1.0
Name: met_threshold, dtype: float64

In [7]:
df_predicted = df.copy()

In [8]:
features = df_predicted.drop('lag_time', axis=1)
target = df_predicted['lag_time']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

In [9]:
lag_model = Sequential([
    Dense(64, activation='relu', input_shape=(features_train.shape[1],), kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(1)
])

lag_model.compile(optimizer=Adam(0.001), loss='mean_squared_error')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = lag_model.fit(features_train, target_train, 
                    epochs=100, 
                    validation_data=(features_test, target_test),
                    callbacks=[early_stop])

Epoch 1/100


UnimplementedError: Graph execution error:

Detected at node 'mean_squared_error/Cast' defined at (most recent call last):
    File "C:\Users\ssaam\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ssaam\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\ssaam\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\ssaam\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\ssaam\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\ssaam\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\ssaam\anaconda3\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_once()
   

In [10]:
print(target_train.dtype)


object
